# 📊 Análisis Exploratorio del Mercado Laboral LatAm

Este notebook sirve como un punto de partida para explorar los datos recopilados por el scraper de vacantes. Utilizaremos `pandas` para la manipulación de datos y `plotly` para visualizaciones interactivas.

In [1]:
import pandas as pd
import plotly.express as px
import os
from dotenv import load_dotenv
import datetime
# import streamlit as st # NO ES NECESARIO EN JUPYTER

# Asegúrate de que estás en la raíz del proyecto o ajusta la ruta
load_dotenv()

# Importar el cliente de Supabase (asumiendo que los paths están correctos)
try:
    from database.supabase_client import SupabaseClient
except ImportError:
    # Ajusta la ruta si es necesario para el contexto del notebook
    import sys
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
    from database.supabase_client import SupabaseClient

print("Librerías cargadas exitosamente.")

Librerías cargadas exitosamente.


## 1. Cargar Datos desde Supabase

Primero, estableceremos una conexión a Supabase y cargaremos los datos de las vacantes (`jobs`) y las tendencias (`trends`).

In [2]:
supabase_client = SupabaseClient()

# @st.cache_data(ttl=3600) # ESTA LÍNEA DEBE SER COMENTADA O ELIMINADA EN JUPYTER
def load_all_data_for_notebook(): # Renombramos la función para distinguirla de la del dashboard
    print("Cargando datos de trabajos de Supabase...")
    jobs_response = supabase_client.get_jobs(limit=None)
    jobs_data = jobs_response.data if jobs_response and jobs_response.data else []
    df_jobs = pd.DataFrame(jobs_data)
    
    print("Cargando datos de tendencias de Supabase...")
    trends_response = supabase_client.get_trends(limit=None)
    trends_data = trends_response.data if trends_response and trends_response.data else []
    df_trends = pd.DataFrame(trends_data)

    # Convertir columnas de fecha
    if 'posted_date' in df_jobs.columns: df_jobs['posted_date'] = pd.to_datetime(df_jobs['posted_date'], errors='coerce')
    if 'scraped_at' in df_jobs.columns: df_jobs['scraped_at'] = pd.to_datetime(df_jobs['scraped_at'], errors='coerce')
    if 'date' in df_trends.columns: df_trends['date'] = pd.to_datetime(df_trends['date'], errors='coerce')
    
    # Limpiar posibles zonas horarias para consistencia
    for col in ['posted_date', 'scraped_at']:
        if col in df_jobs.columns and df_jobs[col].dt.tz is not None:
            df_jobs[col] = df_jobs[col].dt.tz_localize(None)
    if 'date' in df_trends.columns and df_trends['date'].dt.tz is not None:
        df_trends['date'] = df_trends['date'].dt.tz_localize(None)

    # Procesar habilidades anidadas si existen
    if 'skills' in df_jobs.columns and not df_jobs['skills'].empty:
        all_skills_flat = []
        for idx, row in df_jobs.iterrows():
            if row['skills']:
                for skill_entry in row['skills']:
                    all_skills_flat.append({
                        'job_id': row['id'],
                        'skill_name': skill_entry.get('skill_name'),
                        'skill_category': skill_entry.get('skill_category'),
                        'posted_date': row['posted_date'],
                        'country': row['country'],
                        'sector': row['sector'],
                    })
        df_skills = pd.DataFrame(all_skills_flat)
    else:
        df_skills = pd.DataFrame()

    print(f"Jobs cargados: {len(df_jobs)} registros")
    print(f"Skills cargados: {len(df_skills)} registros")
    print(f"Tendencias cargados: {len(df_trends)} registros")

    return df_jobs, df_skills, df_trends

df_jobs, df_skills, df_trends = load_all_data_for_notebook()

2025-12-10 23:48:29,350 - INFO - SupabaseClient inicializado.


Cargando datos de trabajos de Supabase...


2025-12-10 23:48:30,164 - INFO - HTTP Request: GET https://bmkrrxsiltbeoiqgonac.supabase.co/rest/v1/jobs?select=%2A%2Cskills%28%2A%29&order=scraped_at.desc "HTTP/2 200 OK"
2025-12-10 23:48:30,351 - INFO - HTTP Request: GET https://bmkrrxsiltbeoiqgonac.supabase.co/rest/v1/trends?select=%2A&order=date.desc "HTTP/2 200 OK"


Cargando datos de tendencias de Supabase...
Jobs cargados: 20 registros
Skills cargados: 10 registros
Tendencias cargados: 10 registros


## 2. Inspección Inicial de Datos

In [3]:
print("### DataFrame de Vacantes (df_jobs) - Primeras 5 filas ###")
display(df_jobs.head())

print("\n### Información de df_jobs ###")
df_jobs.info()

print("\n### DataFrame de Habilidades (df_skills) - Primeras 5 filas ###")
display(df_skills.head())

print("\n### Información de df_skills ###")
df_skills.info()

print("\n### DataFrame de Tendencias (df_trends) - Primeras 5 filas ###")
display(df_trends.head())

print("\n### Información de df_trends ###")
df_trends.info()

### DataFrame de Vacantes (df_jobs) - Primeras 5 filas ###


,id,job_id,source_platform,title,company_name,location,country,job_type,seniority_level,sector,...,source_url,scraped_at,is_active,company_id,company_size,company_industry,company_hq_country,company_type,company_website,skills
0,ae685ac8-4436-489d-adf6-3842471dce7c,4326575835,LinkedIn,Senior DevOps - 100% remoto .,Talent Connect,Greater Buenos Aires,Argentina,Remote,Senior,Software Development,...,https://ar.linkedin.com/jobs/view/senior-devop...,2025-12-10 21:05:12.982282,True,39f6c0ce-9997-4380-a259-c4c95316a1da,None,None,None,None,None,[{'id': '9f955090-f350-40ee-bde8-49ebb42767a7'...
1,727dfb48-8551-4818-a49b-0c94ed4c7d2e,4326614291,LinkedIn,DevOps con Observability Sr+ - Argentina.,Talent Connect,Greater Buenos Aires,Argentina,Other,Senior,Data Analytics & BI,...,https://ar.linkedin.com/jobs/view/devops-con-o...,2025-12-10 21:05:12.492540,True,39f6c0ce-9997-4380-a259-c4c95316a1da,None,None,None,None,None,[]
2,4da89349-edb0-472b-9149-9e9f8b1fb56f,4342812160,LinkedIn,Lead DevOps Engineer,Search Atlas,Greater Buenos Aires,Argentina,Other,Lead / Manager,Software Development,...,https://ar.linkedin.com/jobs/view/lead-devops-...,2025-12-10 21:05:11.966927,True,6e69d3e3-05dc-40da-9f78-f90378877ccf,None,None,None,None,None,[]
3,5769aa61-a387-4568-b4f4-b56942bb913c,4349473953,LinkedIn,QA Automation Engineer,a1qa,Greater Buenos Aires,Argentina,Other,Mid,Future Of Work,...,https://ar.linkedin.com/jobs/view/qa-automatio...,2025-12-10 21:05:11.475208,True,4c62fdde-286e-4fb3-af9a-fb86a3c4223d,None,None,None,None,None,[]
4,f0655b74-efdc-43be-9301-581055bbd474,4326741381,LinkedIn,DevOps con Observability Sr+ - Remote 100%,Talent Connect,Greater Buenos Aires,Argentina,Remote,Senior,Data Analytics & BI,...,https://ar.linkedin.com/jobs/view/devops-con-o...,2025-12-10 21:05:11.001927,True,39f6c0ce-9997-4380-a259-c4c95316a1da,None,None,None,None,None,[]



### Información de df_jobs ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  20 non-null     object        
 1   job_id              20 non-null     object        
 2   source_platform     20 non-null     object        
 3   title               20 non-null     object        
 4   company_name        20 non-null     object        
 5   location            20 non-null     object        
 6   country             20 non-null     object        
 7   job_type            20 non-null     object        
 8   seniority_level     20 non-null     object        
 9   sector              20 non-null     object        
 10  role_category       0 non-null      object        
 11  description         20 non-null     object        
 12  requirements        20 non-null     object        
 13  salary_range        

,job_id,skill_name,skill_category,posted_date,country,sector
0,ae685ac8-4436-489d-adf6-3842471dce7c,Azure,Cloud/DevOps,2025-12-11,Argentina,Software Development
1,ae685ac8-4436-489d-adf6-3842471dce7c,Kubernetes,Cloud/DevOps,2025-12-11,Argentina,Software Development
2,15e9529b-330c-4103-a853-9c69e9be5b48,Ruby,Programming Language,2025-12-08,Argentina,Software Development
3,7e07ebf3-d611-474e-ab58-1a93ac10e3b7,GCP,Cloud/DevOps,2025-12-09,Argentina,Software Development
4,b8a1b9d6-1809-4f8a-9fdb-aac44f6da3cb,Kotlin,Programming Language,2025-12-10,Argentina,Software Development



### Información de df_skills ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   job_id          10 non-null     object        
 1   skill_name      10 non-null     object        
 2   skill_category  10 non-null     object        
 3   posted_date     10 non-null     datetime64[ns]
 4   country         10 non-null     object        
 5   sector          10 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 612.0+ bytes

### DataFrame de Tendencias (df_trends) - Primeras 5 filas ###


,id,date,metric_name,metric_value,count,sector,country,created_at
0,89607143-7369-47a8-9761-9863a4417a2f,2025-12-10,most_demanded_skill,Kotlin,4,None,None,2025-12-11T01:45:16.508734+00:00
1,9503e211-cdc2-4a10-b1b0-68adc983626a,2025-12-10,most_demanded_skill,Go,1,None,None,2025-12-11T01:45:16.692901+00:00
2,0bd59d41-2ea6-4881-aabd-31a9a4c9d8ca,2025-12-10,most_demanded_skill,Java,1,None,None,2025-12-11T01:45:16.854111+00:00
3,570dcfb2-d212-443d-b157-ab2159c7dee2,2025-12-10,growing_skill,Kotlin (+400.00%),4,None,None,2025-12-11T01:45:17.350151+00:00
4,2e00f33b-2f9f-4295-92fd-e01ede7d41f6,2025-12-10,growing_skill,Java (+100.00%),1,None,None,2025-12-11T01:45:17.59519+00:00



### Información de df_trends ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            10 non-null     object        
 1   date          10 non-null     datetime64[ns]
 2   metric_name   10 non-null     object        
 3   metric_value  10 non-null     object        
 4   count         10 non-null     int64         
 5   sector        2 non-null      object        
 6   country       0 non-null      object        
 7   created_at    10 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 772.0+ bytes


## 3. Visualizaciones Clave

Exploremos algunas visualizaciones para entender mejor el mercado laboral.

### 3.1. Vacantes por País

In [4]:
if not df_jobs.empty and 'country' in df_jobs.columns:
    country_counts = df_jobs['country'].value_counts().reset_index()
    country_counts.columns = ['País', 'Número de Vacantes']
    fig = px.bar(country_counts.head(10), x='Número de Vacantes', y='País', orientation='h',
                 title='Top 10 Países con Más Vacantes',
                 color_continuous_scale=px.colors.sequential.Plasma)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("No hay datos de país para visualizar.")

### 3.2. Vacantes por Sector

In [5]:
if not df_jobs.empty and 'sector' in df_jobs.columns:
    sector_counts = df_jobs['sector'].value_counts().reset_index()
    sector_counts.columns = ['Sector', 'Número de Vacantes']
    fig = px.pie(sector_counts.head(5), values='Número de Vacantes', names='Sector',
                 title='Distribución de Vacantes por Sector (Top 5)',
                 hole=0.3)
    fig.show()
else:
    print("No hay datos de sector para visualizar.")

### 3.3. Habilidades Más Demandadas (Basado en datos crudos de `jobs`)

Aquí visualizamos las habilidades más frecuentes directamente de las vacantes, para ver la demanda general.

In [6]:
if not df_skills.empty and 'skill_name' in df_skills.columns:
    demanded_skills = df_skills['skill_name'].value_counts().reset_index()
    demanded_skills.columns = ['Habilidad', 'Frecuencia']
    fig = px.bar(demanded_skills.head(15), x='Frecuencia', y='Habilidad', orientation='h',
                 title='Top 15 Habilidades Más Mencionadas en Vacantes',
                 color_continuous_scale=px.colors.sequential.Blues)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("No hay datos de habilidades para visualizar.")

### 3.4. Roles Más Demandados (Basado en datos crudos de `jobs`)

Similar a las habilidades, podemos ver los roles que más aparecen en los títulos de las vacantes.

In [7]:
if not df_jobs.empty and 'title' in df_jobs.columns:
    # Una simplificación básica de roles para este análisis
    def simplify_title_for_roles(title):
        title_lower = str(title).lower()
        if 'software engineer' in title_lower or 'ingeniero de software' in title_lower or 'desarrollador' in title_lower: return 'Software Engineer/Developer'
        if 'data scientist' in title_lower or 'científico de datos' in title_lower: return 'Data Scientist'
        if 'product manager' in title_lower or 'gerente de producto' in title_lower: return 'Product Manager'
        if 'devops' in title_lower: return 'DevOps Engineer'
        if 'frontend' in title_lower: return 'Frontend Developer'
        if 'backend' in title_lower: return 'Backend Developer'
        if 'full stack' in title_lower: return 'Full Stack Developer'
        return 'Other/General'

    df_jobs['simplified_role'] = df_jobs['title'].apply(simplify_title_for_roles)
    role_counts = df_jobs['simplified_role'].value_counts().reset_index()
    role_counts.columns = ['Rol', 'Número de Vacantes']
    fig = px.bar(role_counts.head(10), x='Número de Vacantes', y='Rol', orientation='h',
                 title='Top 10 Roles Más Comunes',
                 color_continuous_scale=px.colors.sequential.Oranges)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("No hay datos de títulos de vacantes para analizar roles.")

### 3.5. Tendencias de Habilidades (Desde la tabla `trends`)

Aquí usamos los datos pre-calculados de la tabla `trends` para ver las habilidades en crecimiento.

In [8]:
if not df_trends.empty:
    # Tomar las tendencias más recientes (del último día de análisis)
    latest_analysis_date = df_trends['date'].max()
    latest_trends = df_trends[df_trends['date'] == latest_analysis_date]

    growing_skills_trend = latest_trends[latest_trends['metric_name'] == 'growing_skill'].sort_values('count', ascending=False)
    
    if not growing_skills_trend.empty:
        fig = px.bar(growing_skills_trend.head(15), x='count', y='metric_value', orientation='h',
                     title=f'Top 15 Habilidades en Crecimiento (al {latest_analysis_date.strftime("%Y-%m-%d")})',
                     labels={'count': 'Vacantes Actuales', 'metric_value': 'Habilidad (Tasa de Crecimiento)'},
                     color_continuous_scale=px.colors.sequential.Greens)
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
        fig.show()
    else:
        print("No hay datos de habilidades en crecimiento en la tabla de tendencias.")
else:
    print("La tabla de tendencias está vacía. Ejecuta el análisis de tendencias con `python main.py --analyze-trends`.")

## 4. Conclusión

Este notebook proporciona una visión básica del mercado laboral LatAm basada en los datos scrapeados. Puedes extender este análisis para:
- Segmentar por país o sector.
- Comparar tendencias a lo largo de diferentes períodos.
- Realizar un análisis de texto más profundo en descripciones de vacantes.
- Integrar otras fuentes de datos.